In [1]:
import numpy as np
import pandas as pd
import pickle as cPickle

load the salary matrix and similarity matrix for model

In [2]:
with open('salary_lower_bound_matrix.p','rb') as f:
    df_low=cPickle.load(f)
with open('salary_upper_bound_matrix.p','rb') as f:
    df_upper=cPickle.load(f)
with open('job_similarity_matrix.p','rb') as f:
    M=cPickle.load(f)
with open('comany_similarity_matrix.p','rb') as f:
    N=cPickle.load(f)

In [3]:
## this function used for evaluation and save results
import time
def evaluate(algo,testset,algoname,savefilename):
    import numpy as np
    import time
    f=open(savefilename,'ab')
    val_n = 0
    val_rmse = 0
    val_mae = 0
    for u, i, r in testset:
        est = algo.predict(int(u), int(i), r_ui=r)
        val_rmse = val_rmse + (r - est[3]) ** 2
        val_mae = val_mae + abs(r - est[3])
        val_n+=1
    val_rmse = np.sqrt(val_rmse / val_n)
    val_mae = val_mae / val_n
    ct=time.ctime()
    f.write('%s, val rmse is:%.6f,val mae is:%.6f\t%s\n'%(algoname,val_rmse,val_mae,ct))
    f.close()
    print('%s, val rmse is:%.6f,val mae is:%.6f'%(algoname,val_rmse,val_mae))
    return val_rmse, val_mae

split and prepare the data for 5 cross validation 

In [4]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1,5),line_format='user item rating',sep='\t')
data1=Dataset.load_from_df(df_low,reader)
data1.split(5)
reader = Reader(rating_scale=(1,5),line_format='user item rating',sep='\t')
data2=Dataset.load_from_df(df_upper,reader)
data2.split(5)

train the model

In [5]:
import SalaryBenchmark_MF as mf
import SalaryBenchmark_MF_SideMatrix as sm

f1='result_lower_bound.txt'
f2='result_upper_bound.txt'

for fold,(trainset,testset) in enumerate(data1.folds()):
    aux_pu,aux_qi=sm.SalaryBenchamrk_SVD_SideMatrix(trainset,M,N,reg_su=1e-4,reg_si=1e-4,reg_t=1e-4,reg_l=1e-4)
    algo1=mf.SalaryBenchmark_SVD(n_factors=5, n_epochs=100)
    algo1.train(trainset,aux_pu,aux_qi)
    evaluate(algo1,testset,'HSBMF',f1)

        
for fold,(trainset,testset) in enumerate(data2.folds()):
    aux_pu,aux_qi=sm.SalaryBenchamrk_SVD_SideMatrix(trainset,M,N,reg_su=1e-4,reg_si=1e-4,reg_t=1e-4,reg_l=1e-4)
    algo2=mf.SalaryBenchmark_SVD(n_factors=5, n_epochs=100)
    algo2.train(trainset,aux_pu,aux_qi)
    evaluate(algo2,testset,'HSBMF',f2)


HSBMF, val rmse is:0.774426,val mae is:0.593820
HSBMF, val rmse is:0.784219,val mae is:0.603323
HSBMF, val rmse is:0.785391,val mae is:0.603090
HSBMF, val rmse is:0.781022,val mae is:0.599359
HSBMF, val rmse is:0.779051,val mae is:0.597341
HSBMF, val rmse is:0.778294,val mae is:0.590092
HSBMF, val rmse is:0.777658,val mae is:0.592203
HSBMF, val rmse is:0.775638,val mae is:0.592259
HSBMF, val rmse is:0.775086,val mae is:0.588755
HSBMF, val rmse is:0.771332,val mae is:0.587588
